In [1]:
import pandas as pd
import os
import sys
import numpy as np

In [2]:
src_path = os.path.dirname(os.path.realpath('__file__'))
data_path = os.path.join(os.path.dirname(src_path), 'data')

### Liste des acteurs

In [3]:
name_basics_file = os.path.join(data_path, 'name.basics.tsv')
title_akas_file = os.path.join(data_path, 'title.akas.tsv')
title_basics_file = os.path.join(data_path, 'title.basics.tsv')
title_principals_file = os.path.join(data_path, 'title.principals.tsv')
title_ratings_file = os.path.join(data_path, 'title.ratings.tsv')

df_name_basics = pd.read_table(name_basics_file, nrows=100000)
df_title_akas = pd.read_table(title_akas_file, nrows=100000)
df_title_basics = pd.read_table(title_basics_file, nrows=100000)
df_title_principals = pd.read_table(title_principals_file, nrows=100000)
df_title_ratings = pd.read_table(title_ratings_file, nrows=100000)

In [13]:
df_principals = df_title_principals.loc[df_title_principals['category'].isin(['actor', 'actress'])].filter(['tconst', 'nconst']).drop_duplicates()

df_principals_actor_assoc = pd.merge(df_principals, df_name_basics, how='inner', on='nconst')[['tconst', 'nconst', 'primaryName']].drop_duplicates()
df_principals_film_actor_assoc = pd.merge(df_principals_actor_assoc, df_title_basics, on='tconst')[['originalTitle', 'primaryName']].drop_duplicates()

df_actors_groupes_by_films = df_principals_film_actor_assoc.groupby(['originalTitle', 'primaryName'])

#df_actors_groupes_by_films.first()

## Question 2

In [22]:
# get only movies distriuted in US
df_akas_us = df_title_akas.loc[df_title_akas['region'] == "US"].filter(['titleId', 'title', 'region'])
df_akas_us.rename(columns = {'title': 'title_us', 'region': 'region_us'}, inplace = True)

# get only movies distriuted in FR
df_akas_fr = df_title_akas.loc[df_title_akas['region'] == "FR"].filter(['titleId', 'title', 'region'])
df_akas_fr.rename(columns = {'title': 'title_fr', 'region': 'region_fr'}, inplace = True)

# merge movies distributed in US and in FR
df_akas_merged = pd.merge(df_akas_us, df_akas_fr, how='left', on='titleId')

# create new column "is_title_fr" to describe if movie has english title or only french title
df_akas_merged['is_title_fr'] = np.where(df_akas_merged['title_fr'].isnull(), False, True)

# remove US title if FR title exists
df_akas_merged['title_us'] = np.where(df_akas_merged['title_fr'].notnull(), '', df_akas_merged['title_us'])

# merge with rating
df_rating_akas = pd.merge(df_akas_merged, df_title_ratings, how='inner', right_on='tconst', left_on='titleId')

# remove columns
df_rating_akas = df_rating_akas.filter(['titleId', 'title_fr', 'title_us', 'is_title_fr', 'averageRating'])

### Question 3

In [6]:
new_df = pd.DataFrame(df_title_basics.genres.str.split(',').tolist(), index = df_title_basics.tconst).stack()
new_df = new_df.reset_index([0, 'tconst'])
new_df.columns = ['tconst', 'genre']

In [7]:
merge_genre_rating = pd.merge(new_df, title_ratings_file, how = 'inner', on = 'tconst')
merge_genre_rating

TypeError: Can only merge Series or DataFrame objects, a <class 'str'> was passed